ローカル環境で作成したSARIMAXモデルです。
Google Colaboratoryの場合、pandas,numpyのインストール時にエラー出て再起動を求められますが、そのまま実行できます。

In [1]:
!pip install --force-reinstall pandas==1.5.3
!pip install --force-reinstall numpy==1.26.4

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 573.7 kB/s eta 0:00:00
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 7.5 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.2
    Uninstalling pytz-2024.2:
      Successfully uninstalled pytz-2024.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
 

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.11.4 requires pandas>=2.1.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.13.6 requires pandas<3.0.0,>=2.1.0, but you have pandas 1.5.3 which is incompatible.
xarray 2024.9.0 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import subprocess

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=f459ecefe4c658c70a9cf3615245ecb799400fb3d4696c198837a838e2af93bc
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib
japanize_matplotlib has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00
catboost has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
optuna has been installed.
geolib has been installed.
Directory and file for OpenCL vendors have been created.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns
import pickle
from catboost import CatBoostRegressor, Pool
import os

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

from utils import (
                   to_cat_col,
                   VotingModel,
                   check_dataframe_elements
                   )

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

sarima_model_path="/content/drive/MyDrive/yuto/project_directory/models/model_sarimax.pkl"

In [5]:
df=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv",index_col=0)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '単位面積あたりの取引価格_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]

In [6]:
df_all_year=df.groupby("取引時点")[f"{PRICE_PER_AREA_LOG}"].mean().rename(f"{PRICE_PER_AREA_LOG}_mean")
# 最後のインデックスに続けて新しい値を追加
last_index = df_all_year.index[-1]  # 最後のインデックスを取得
print(df_all_year.index)

Float64Index([2005.75, 2005.99, 2006.25,  2006.5, 2006.75, 2006.99, 2007.25,
               2007.5, 2007.75, 2007.99, 2008.25,  2008.5, 2008.75, 2008.99,
              2009.25,  2009.5, 2009.75, 2009.99, 2010.25,  2010.5, 2010.75,
              2010.99, 2011.25,  2011.5, 2011.75, 2011.99, 2012.25,  2012.5,
              2012.75, 2012.99, 2013.25,  2013.5, 2013.75, 2013.99, 2014.25,
               2014.5, 2014.75, 2014.99, 2015.25,  2015.5, 2015.75, 2015.99,
              2016.25,  2016.5, 2016.75, 2016.99, 2017.25,  2017.5, 2017.75,
              2017.99, 2018.25,  2018.5, 2018.75, 2018.99, 2019.25,  2019.5,
              2019.75, 2019.99, 2020.25,  2020.5, 2020.75, 2020.99, 2021.25,
               2021.5, 2021.75, 2021.99, 2022.25,  2022.5, 2022.75, 2022.99,
              2023.25,  2023.5, 2023.75, 2023.99],
             dtype='float64', name='取引時点')


In [7]:
# モデルの読み込み
#model_path = r"C:\Users\yuto2\OneDrive\ドキュメント\Pythonフォルダ\Nishika\中古マンション価格予測_2024夏の部\Nishika1st_動けばOK\model_sarimax.pkl"
# Pandasのバージョンを確認
print(pd.__version__)

# Pandasのバージョンが2.0以降の場合、以下を試してください。
try:
    with open(sarima_model_path, 'rb') as file:
        SARIMA_model_fit = pickle.load(file)
except ModuleNotFoundError:
    # Pandas 1.5.3を強制的に再インストール
    !pip install --force-reinstall pandas==1.5.3
    !pip install --force-reinstall numpy==1.26.4
    # Pandasをインポートし直す
    import pandas as pd
    with open(sarima_model_path, 'rb') as file:
        SARIMA_model_fit = pickle.load(file)


print("モデルの読み込みが完了しました。")

# 予測の実行
SARIMA_pred = SARIMA_model_fit.get_forecast(steps=2)
SARIMA_pred_df = SARIMA_pred.summary_frame()

# 予測結果の表示
print(SARIMA_pred_df)

1.5.3
モデルの読み込みが完了しました。
単位面積あたりの取引価格_log_mean      mean   mean_se  mean_ci_lower  mean_ci_upper
72                     5.629596  0.014408       5.601356       5.657837
73                     5.634549  0.019483       5.596363       5.672735


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [8]:
#print(df_all_year)
df_all_year.loc[2023.75]=SARIMA_pred_df["mean"][72]
df_all_year.loc[2023.99]=SARIMA_pred_df["mean"][73]
print(df_all_year)

取引時点
2005.75    5.603129
2005.99    5.600274
2006.25    5.610975
2006.50    5.505956
2006.75    5.511552
             ...   
2022.99    5.617195
2023.25    5.624689
2023.50    5.620718
2023.75    5.629596
2023.99    5.634549
Name: 単位面積あたりの取引価格_log_mean, Length: 74, dtype: float64
